In [1]:
from dataclasses import dataclass, field
import re

import torch
from piq import multi_scale_ssim as ms_ssim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import euclidean_distances
import cv2
import os
import random
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot

from src.metrics import *
from src.utils import *
from src.datasets import ExcavatorDataset
from src.config import TRANSFORMER, ROOT, DEVICE
from models.Segmentation import DeepLabV3Model, DeepLabV3PlusModel, PyramidAttentionNetworkModel, UNetModel

In [2]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', plot=False)
train_paths = [os.path.basename(path) for *_, path in train_dataset]
val_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='validation', plot=False)
val_paths = [os.path.basename(path) for *_, path in val_dataset]
cls_colors = train_dataset.class_colors

C:\Users\vunha\Documents\Uni_Infos\Bachelorarbeit\workspace\similarity_metrics_of_images\src\datasets.py:313: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


# Load IoUs

In [3]:
# DeepLabV3
dlv3 =DeepLabV3Model(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3_HybridFocalDiceLoss.pt')

# DeepLabV3Plus
dlv3p = DeepLabV3PlusModel(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3Plus_HybridFocalDiceLoss.pt')

# UNet
unet = UNetModel(model_path=f'{ROOT}/models/torch_model_files/UNet_HybridFocalDiceLoss.pt')

# Pyramid Attention Network
pan = PyramidAttentionNetworkModel(model_path=f'{ROOT}/models/torch_model_files/PyramidAttentionNetwork_HybridFocalDiceLoss.pt')

# Pass a model to its corresponding ioU file
iou_paths = {
    dlv3: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3.h5'
    },
    dlv3p: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3p.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3p.h5'
    },
    unet: {
        'train': f'{ROOT}/res/model_performance/train_iou_unet.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_unet.h5'
    },
    pan: {
        'train': f'{ROOT}/res/model_performance/train_iou_pan.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_pan.h5'
    }
}

2024-12-08 20:40:53,135 - DeepLabV3 - INFO - Device used for model: cuda
2024-12-08 20:40:53,354 - DeepLabV3Plus - INFO - Device used for model: cuda
2024-12-08 20:40:53,624 - UNet - INFO - Device used for model: cuda
2024-12-08 20:40:53,881 - PyramidAttentionNetwork - INFO - Device used for model: cuda


# 1. VLAD

In [ ]:
k_means_models = [
    model for model in os.listdir(rf'{ROOT}/models/pickle_model_files') if 'k_means' in model
]

for seg_models in iou_paths:
    model_name = seg_models.model.__class__.__name__
    iou_train = load_hdf5(iou_paths[seg_models]['train'])
    iou_val = load_hdf5(iou_paths[seg_models]['val'])
    for model in k_means_models:
        k_model = model.replace('.pkl', '')
        similarity_scores = []
        euclidean_similarity = []
        iou_diff = []
        train_vlad_data = load_hdf5(f'{ROOT}/res/vlad/train/{model[:-4]}.h5')
        val_vlad_data = load_hdf5(f'{ROOT}/res/vlad/validation/{model[:-4]}.h5')
        for train_path in train_paths:
            for val_path in val_paths:
                vlad_1 = train_vlad_data[train_path]
                vlad_2 = val_vlad_data[val_path]
                similarity_scores.append(cosine_similarity([vlad_1], [vlad_2])[0][0])
                euclidean_similarity.append(1.0 / (1.0 + euclidean_distances([vlad_1], [vlad_2])[0][0]))
                iou_train_img = iou_train[train_path]
                iou_val_img = iou_val[val_path]
                iou_diff.append(abs(iou_train_img - iou_val_img))
        plot_boxplot_with_regression(y=np.array(similarity_scores),
                                     x=np.array(iou_diff),
                                     x_lim=(0, 1),
                                     y_lim=(-1, 1),
                                     num_bins=15,
                                     title=f"{model_name} vs {k_model} - IoU diff vs Similarity",
                                     y_label="Similarity Score",
                                     x_label="IoU Difference",
                                     save_fig_path=f"{ROOT}/res/plots/vlad/{model_name}_{k_model}_iou_vs_similarity.png")
        plot_scatter_with_regression(y=np.array(similarity_scores),
                                     x=np.array(iou_diff),
                                     x_lim=(0, 1),
                                        y_lim=(-1, 1),
                                        title=f"{model_name} vs {k_model} - IoU diff vs Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/vlad/{model_name}_{k_model}_iou_vs_similarity_scatter.png")
        plot_boxplot_with_regression(y=np.array(euclidean_similarity),
                                        x=np.array(iou_diff),
                                        x_lim=(0, 1),
                                        y_lim=(min(euclidean_similarity), max(euclidean_similarity)),
                                        num_bins=15,
                                        title=f"{model_name} vs {k_model} - IoU diff vs Euclidean Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/vlad/{model_name}_{k_model}_iou_vs_euclidean_similarity.png")
        plot_scatter_with_regression(y=np.array(euclidean_similarity),
                                        x=np.array(iou_diff),
                                        x_lim=(0, 1),
                                        y_lim=(min(euclidean_similarity), max(euclidean_similarity)),
                                        title=f"{model_name} vs {k_model} - IoU diff vs Euclidean Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/vlad/{model_name}_{k_model}_iou_vs_euclidean_similarity_scatter.png")





# 2. Fisher Vector

In [ ]:
gmm_models = [
    model for model in os.listdir(rf'{ROOT}/models/pickle_model_files') if 'gmm' in model
]


for seg_models in iou_paths:
    model_name = seg_models.model.__class__.__name__
    iou_train = load_hdf5(iou_paths[seg_models]['train'])
    iou_val = load_hdf5(iou_paths[seg_models]['val'])
    for model in gmm_models:
        g_model = model.replace('.pkl', '')
        similarity_scores = []
        euclidean_similarity = []
        iou_diff = []
        train_fisher_data = load_hdf5(f'{ROOT}/res/fisher/train/{model[:-4]}.h5')
        val_fisher_data = load_hdf5(f'{ROOT}/res/fisher/validation/{model[:-4]}.h5')
        for train_path in train_paths:
            for val_path in val_paths:
                fisher_1 = train_fisher_data[train_path]
                fisher_2 = val_fisher_data[val_path]
                similarity_scores.append(cosine_similarity([fisher_1], [fisher_2])[0][0])
                euclidean_similarity.append(1.0 / (1.0 + euclidean_distances([fisher_1], [fisher_2])[0][0]))
                iou_train_img = iou_train[train_path]
                iou_val_img = iou_val[val_path]
                iou_diff.append(abs(iou_train_img - iou_val_img))
        plot_boxplot_with_regression(y=np.array(similarity_scores),
                                     x=np.array(iou_diff),
                                     x_lim=(0, 1),
                                     y_lim=(-1, 1),
                                     num_bins=15,
                                     title=f"{model_name} vs {g_model} - IoU diff vs Similarity",
                                     y_label="Similarity Score",
                                     x_label="IoU Difference",
                                     save_fig_path=f"{ROOT}/res/plots/fisher/{model_name}_{g_model}_iou_vs_similarity.png")
        plot_scatter_with_regression(y=np.array(similarity_scores),
                                     x=np.array(iou_diff),
                                     x_lim=(0, 1),
                                        y_lim=(-1, 1),
                                        title=f"{model_name} vs {g_model} - IoU diff vs Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/fisher/{model_name}_{g_model}_iou_vs_similarity_scatter.png")
        plot_boxplot_with_regression(y=np.array(euclidean_similarity),
                                        x=np.array(iou_diff),
                                        x_lim=(0, 1),
                                        y_lim=(min(euclidean_similarity), max(euclidean_similarity)),
                                        num_bins=15,
                                        title=f"{model_name} vs {g_model} - IoU diff vs Euclidean Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/fisher/{model_name}_{g_model}_iou_vs_euclidean_similarity.png")
        plot_scatter_with_regression(y=np.array(euclidean_similarity),
                                        x=np.array(iou_diff),
                                        x_lim=(0, 1),
                                        y_lim=(min(euclidean_similarity), max(euclidean_similarity)),
                                        title=f"{model_name} vs {g_model} - IoU diff vs Euclidean Similarity",
                                        y_label="Similarity Score",
                                        x_label="IoU Difference",
                                        save_fig_path=f"{ROOT}/res/plots/fisher/{model_name}_{g_model}_iou_vs_euclidean_similarity_scatter.png")




In [5]:
train_dataset_transformed = ExcavatorDataset(return_type='image+mask+path', purpose='train', plot=False, transform=TRANSFORMER)
val_dataset_transformed = ExcavatorDataset(return_type='image+mask+path', purpose='validation', plot=False, transform=TRANSFORMER)

C:\Users\vunha\Documents\Uni_Infos\Bachelorarbeit\workspace\similarity_metrics_of_images\src\datasets.py:313: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


# Transform datasets for SSIM and MS-SSIM

# 3. SSIM

In [22]:
for seg_models in iou_paths:
    model_name = seg_models.model.__class__.__name__
    iou_train = load_hdf5(iou_paths[seg_models]['train'])
    iou_val = load_hdf5(iou_paths[seg_models]['val'])
    ssim_scores = []
    iou_diff = []

    for train_img, *_, train_path in train_dataset_transformed:
        for j, (val_img, *_, val_path) in enumerate(val_dataset_transformed):
            ssim_scores.append(ssim(train_img.unsqueeze(0), val_img.unsqueeze(0)).cpu().numpy())
            iou_train_img = iou_train[os.path.basename(train_path)]
            iou_val_img = iou_val[os.path.basename(val_path)]
            iou_diff.append(abs(iou_train_img - iou_val_img))
            if j == 10:
                break
    plot_boxplot_with_regression(y=np.array(ssim_scores),
                                 x=np.array(iou_diff),
                                 x_lim=(0, 1),
                                 y_lim=(-1, 1),
                                 num_bins=15,
                                 title=f"{model_name} - IoU diff vs SSIM",
                                 y_label="SSIM Score",
                                 x_label="IoU Difference",
                                 save_fig_path=f"{ROOT}/res/plots/ssim/{model_name}_iou_vs_ssim.png")
    plot_scatter_with_regression(y=np.array(ssim_scores),
                                    x=np.array(iou_diff),
                                    x_lim=(0, 1),
                                    y_lim=(-1, 1),
                                    title=f"{model_name} - IoU diff vs SSIM",
                                    y_label="SSIM Score",
                                    x_label="IoU Difference",
                                    save_fig_path=f"{ROOT}/res/plots/ssim/{model_name}_iou_vs_ssim_scatter.png")

KeyboardInterrupt: 

# 4. MS-SSIM

In [21]:
for seg_models in iou_paths:
    model_name = seg_models.model.__class__.__name__
    iou_train = load_hdf5(iou_paths[seg_models]['train'])
    iou_val = load_hdf5(iou_paths[seg_models]['val'])
    mssim_scores = []
    iou_diff = []

    for train_img, *_, train_path in train_dataset_transformed:
        for j, (val_img, *_, val_path) in enumerate(val_dataset_transformed):
            mssim_scores.append(ms_ssim(train_img.unsqueeze(0), val_img.unsqueeze(0)).cpu().numpy())
            iou_train_img = iou_train[os.path.basename(train_path)]
            iou_val_img = iou_val[os.path.basename(val_path)]
            iou_diff.append(abs(iou_train_img - iou_val_img))
            if j == 10:
                break
    plot_boxplot_with_regression(y=np.array(mssim_scores),
                                 x=np.array(iou_diff),
                                 x_lim=(0, 1),
                                 y_lim=(-1, 1),
                                 num_bins=15,
                                 title=f"{model_name} - IoU diff vs MS-SSIM",
                                 y_label="MS-SSIM Score",
                                 x_label="IoU Difference",
                                 save_fig_path=f"{ROOT}/res/plots/mssim/{model_name}_iou_vs_mssim.png")
    plot_scatter_with_regression(y=np.array(mssim_scores),
                                    x=np.array(iou_diff),
                                    x_lim=(0, 1),
                                    y_lim=(-1, 1),
                                    title=f"{model_name} - IoU diff vs MS-SSIM",
                                    y_label="MS-SSIM Score",
                                    x_label="IoU Difference",
                                    save_fig_path=f"{ROOT}/res/plots/ms_sim/{model_name}_iou_vs_mssim_scatter.png")

KeyboardInterrupt: 

# Combination: Fisher + MS-SSIM